<a href="https://colab.research.google.com/github/srujaan/DL-with-PyTorch/blob/master/find_units.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
# "t_c" is temperature in celcius
# "t_u" is temperature are unknown units
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [3]:
# model
def model(t_u, w, b):
  return w * t_u + b

In [4]:
# loss function
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [9]:
# initialize w,b parameteres
w = torch.ones(())
b = torch.ones(())

In [10]:
t_p = model(t_u, w, b)
t_p

tensor([36.7000, 56.9000, 59.2000, 82.9000, 57.3000, 49.9000, 34.9000, 22.8000,
        49.4000, 61.4000, 69.4000])